In [2]:
# ============================================================
# FINAL STABILIZED VOLLEYBALL ANALYSIS (NO LAG, NO GHOSTS)
# ============================================================

import cv2
import numpy as np
import torch
from ultralytics import YOLO
from collections import deque, Counter

# ------------------------------
# DEVICE & MODEL
# ------------------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
yolo = YOLO("yolov8n.pt").to(DEVICE)

PERSON_ID = 0
BALL_ID = 32

# ------------------------------
# MANUAL GEOMETRY (EDIT)
# ------------------------------
COURT_POLY = np.array([
    [313, 292],
    [996, 289],
    [1198, 708],
    [125, 707],
], dtype=np.int32)

SPLIT_Y = 350

# ------------------------------
# VIDEO I/O
# ------------------------------
VIDEO_PATH = "/home/naman/Cryptonite-RTP-NamanGoel/Task-0/volleyball_match.mp4"
OUTPUT_PATH = "/home/naman/Cryptonite-RTP-NamanGoel/Task-0/volleyball_match_output_new_2.mp4"

cap = cv2.VideoCapture(VIDEO_PATH)
FPS = int(cap.get(cv2.CAP_PROP_FPS))
W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
FRAME_AREA = W * H

out = cv2.VideoWriter(
    OUTPUT_PATH,
    cv2.VideoWriter_fourcc(*"mp4v"),
    FPS,
    (W, H)
)

# ------------------------------
# COURT MASK
# ------------------------------
def inside_court(cx, cy):
    return cv2.pointPolygonTest(
        COURT_POLY,
        (float(cx), float(cy)),
        False
    ) >= 0

# ------------------------------
# BALL KALMAN (SHORT GAP ONLY)
# ------------------------------
kalman = cv2.KalmanFilter(4, 2)
kalman.measurementMatrix = np.array([[1,0,0,0],[0,1,0,0]], np.float32)
kalman.transitionMatrix = np.array([[1,0,1,0],[0,1,0,1],[0,0,1,0],[0,0,0,1]], np.float32)
kalman.processNoiseCov = np.eye(4, dtype=np.float32) * 0.005

ball_trail = deque(maxlen=30)
ball_missing = 0

# ------------------------------
# PLAYER TRACKER
# ------------------------------
def iou(a, b):
    xA,yA = max(a[0],b[0]), max(a[1],b[1])
    xB,yB = min(a[2],b[2]), min(a[3],b[3])
    inter = max(0,xB-xA)*max(0,yB-yA)
    areaA = (a[2]-a[0])*(a[3]-a[1])
    areaB = (b[2]-b[0])*(b[3]-b[1])
    return inter / (areaA+areaB-inter+1e-6)

players = {}
NEXT_ID = 0
MAX_MISS = 20

team_history = deque(maxlen=30)

# ------------------------------
# MAIN LOOP
# ------------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = yolo(frame, device=DEVICE, verbose=False)

    detections = []
    ball_measure = None
    best_ball_area = 0

    for r in results:
        for b in r.boxes:
            cls = int(b.cls[0])
            x1,y1,x2,y2 = b.xyxy[0].cpu().numpy().astype(int)
            cx,cy = (x1+x2)//2, (y1+y2)//2
            area = (x2-x1)*(y2-y1)

            # ---------------- PERSON ----------------
            if cls == PERSON_ID and b.conf[0] > 0.45:
                if not inside_court(cx, cy):
                    continue
                if area < 2000 or area > 0.15*FRAME_AREA:
                    continue
                detections.append((x1,y1,x2,y2,cx,cy))

            # ---------------- BALL ----------------
            elif cls == BALL_ID and b.conf[0] > 0.12:
                if 50 < area < 0.01*FRAME_AREA and area > best_ball_area:
                    best_ball_area = area
                    ball_measure = np.array([[np.float32(cx)], [np.float32(cy)]])

    # ---------------- PLAYER TRACKING ----------------
    updated = set()
    for d in detections:
        x1,y1,x2,y2,cx,cy = d
        best_id,best_i = None,0
        for pid,p in players.items():
            i = iou((x1,y1,x2,y2), p["bbox"])
            if i > best_i:
                best_i,best_id = i,pid
        if best_i > 0.3:
            players[best_id]["bbox"]=(x1,y1,x2,y2)
            players[best_id]["miss"]=0
            updated.add(best_id)
        else:
            team = "A" if cy > SPLIT_Y else "B"
            players[NEXT_ID]={"bbox":(x1,y1,x2,y2),"team":team,"miss":0}
            updated.add(NEXT_ID)
            NEXT_ID+=1

    for pid in list(players):
        if pid not in updated:
            players[pid]["miss"]+=1
            if players[pid]["miss"]>MAX_MISS:
                del players[pid]

    # ---------------- BALL TRACKING ----------------
    if ball_measure is not None:
        kalman.correct(ball_measure)
        ball_missing = 0
        pred = kalman.predict()
        ball_trail.append((int(pred[0]), int(pred[1])))
    else:
        ball_missing += 1
        if ball_missing < 5:
            pred = kalman.predict()
            ball_trail.append((int(pred[0]), int(pred[1])))

    # ---------------- COUNTER STABILIZATION ----------------
    teamA = sum(1 for p in players.values() if p["team"]=="A")
    teamB = sum(1 for p in players.values() if p["team"]=="B")
    team_history.append((teamA, teamB))

    stable_A = Counter([t[0] for t in team_history]).most_common(1)[0][0]
    stable_B = Counter([t[1] for t in team_history]).most_common(1)[0][0]

    # ---------------- DRAW ----------------
    for pid,p in players.items():
        x1,y1,x2,y2 = p["bbox"]
        color=(0,255,0) if p["team"]=="A" else (255,0,0)
        cv2.rectangle(frame,(x1,y1),(x2,y2),color,2)
        cv2.putText(frame,f"{p['team']}-{pid}",(x1,y1-5),
                    cv2.FONT_HERSHEY_SIMPLEX,0.5,color,2)

    for i in range(1,len(ball_trail)):
        cv2.line(frame,ball_trail[i-1],ball_trail[i],(0,255,255),2)

    cv2.putText(frame,f"Team A: {stable_A}",(20,30),
                cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,0),2)
    cv2.putText(frame,f"Team B: {stable_B}",(20,60),
                cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,0,0),2)

    out.write(frame)

cap.release()
out.release()
print("DONE. Stable output saved:", OUTPUT_PATH)

/tmp/ipykernel_5154/427329750.py:159: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ball_trail.append((int(pred[0]), int(pred[1])))
/tmp/ipykernel_5154/427329750.py:154: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ball_trail.append((int(pred[0]), int(pred[1])))


DONE. Stable output saved: /home/naman/Cryptonite-RTP-NamanGoel/Task-0/volleyball_match_output_new_2.mp4
